# 🚀 İnteraktif JetX Model Eğitmeni (v4.2 - Arayüz Düzeltmesi)
## 🛠️ 1. Sistem Kurulumu ve Projeyi Yükleme

Bu hücre, projeyi GitHub'dan klonlar (veya günceller), gerekli tüm kütüphaneleri yükler ve Colab arayüzü için gerekli eklentileri etkinleştirir.

In [ ]:
import os
import sys
import subprocess

def setup_environment():
    """Clones or updates the repository and sets up the environment."""
    IN_COLAB = 'google.colab' in sys.modules
    if IN_COLAB:
        print("🔌 Enabling ipywidgets extension for Google Colab...")
        from google.colab import output
        output.enable_custom_widget_manager()
        print("✅ ipywidgets extension enabled.")
    
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"

    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)

    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")

    requirements_path = os.path.join(project_dir, 'requirements_enhanced.txt')
    if os.path.exists(requirements_path):
        print("📦 Gerekli kütüphaneler yükleniyor...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", requirements_path])
        print("✅ Kütüphaneler başarıyla yüklendi.")

setup_environment()
print("🎉 Kurulum tamamlandı!")

## 📊 2. MLflow Deney Takibini Başlatma

In [ ]:
from getpass import getpass
from colab.health_check import ColabHealthCheck
import time

NGROK_AUTH_TOKEN = getpass('Lütfen ngrok authtokeninizi girin: ')

health_checker = ColabHealthCheck()
ngrok_ok, error_message = health_checker.check_ngrok_tunnel(NGROK_AUTH_TOKEN)

if ngrok_ok:
    print("✅ ngrok bağlantı testi başarılı.")
    get_ipython().system_raw('mlflow ui --host 0.0.0.0 &')
    time.sleep(5)
    print("\n✅ MLflow arayüzü için public URL oluşturuluyor...")
    print("👇 Aşağıdaki linke tıklayarak deneyleri izleyebilirsiniz:")
    try:
        from pyngrok import ngrok
        public_url = ngrok.connect(5000, "http").public_url
        print(f"--> {public_url}")
    except Exception as e:
        print(f"❌ ngrok tüneli oluşturulamadı. Hata: {e}")
else:
    print(f"\n❌ ngrok ön kontrolü başarısız oldu. Lütfen authtoken'ınızı ve internet bağlantınızı kontrol edin.")
    print(f"   Hata Detayı: {error_message}")

## 🚀 3. Modelleri Seç ve Eğitimi Başlat

In [ ]:
import torch
import ipywidgets as widgets
from IPython.display import display
from src.training.pipeline_manager import PipelineManager
from src.config.settings import get_all_models
import traceback

print("🔍 Eğitilebilir modeller alınıyor...")
try:
    available_models = get_all_models()
    if not available_models:
        print("❌ UYARI: `get_all_models()` fonksiyonu boş bir liste döndürdü. `config.yaml` dosyasında modellerin tanımlı olduğundan emin olun.")
    else:
        print(f"✅ Başarıyla {len(available_models)} model bulundu: {available_models}")
except Exception as e:
    print(f"❌ HATA: `get_all_models()` fonksiyonu çağrılırken bir hata oluştu: {e}")
    traceback.print_exc()
    available_models = []

print("🎨 Arayüz elemanları oluşturuluyor...")
try:
    if available_models:
        model_checkboxes = [widgets.Checkbox(value=False, description=model) for model in available_models]
        start_button = widgets.Button(description=\"Eğitimi Başlat\", button_style='success', icon='rocket')
        output_area = widgets.Output()
        print("✅ Checkbox, button ve output alanı başarıyla oluşturuldu.")
    else:
        model_checkboxes = []
        start_button = None
        output_area = widgets.Output()
        with output_area:
            output_area.clear_output()
            print("‼️ Model bulunamadığı için arayüz oluşturulamadı. Lütfen yukarıdaki logları kontrol edin.")
except Exception as e:
    print(f"❌ HATA: Arayüz elemanları oluşturulurken bir hata oluştu: {e}")
    traceback.print_exc()
    model_checkboxes = []
    start_button = None
    output_area = widgets.Output()

def on_button_clicked(b):
    \"\"\"Callback function for the 'Start Training' button.\"\"\"
    with output_area:
        output_area.clear_output()
        
        if not model_checkboxes:
            print("❌ Modeller yüklenemediği için eğitim başlatılamıyor.")
            return

        selected_models = [cb.description for cb in model_checkboxes if cb.value]
        
        if not selected_models:
            print("⚠️ Lütfen en az bir model seçin!")
            return

        print(f"🚀 Seçilen modellerle eğitim başlatılıyor: {', '.join(selected_models)}")
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if device == 'cuda':
            print(f"✅ GPU bulundu: {torch.cuda.get_device_name(0)}")
        else:
            print("⚠️ GPU bulunamadı. Eğitim CPU ile devam edecek.")
            
        try:
            pipeline_manager = PipelineManager(models_to_train=selected_models, device=device)
            pipeline_manager.run_pipeline()
        except Exception as e:
            print(f"❌ Eğitim sırasında beklenmedik bir hata oluştu: {e}")
            traceback.print_exc()

def display_training_ui():
    \"\"\"Creates and displays the interactive UI for model training.\"\"\"
    print("🖥️ Arayüz oluşturuluyor ve görüntüleniyor...")
    
    if not start_button or not model_checkboxes:
        print("❌ Arayüz elemanları eksik olduğu için görüntülenemiyor.")
        display(output_area)
        return

    # Buton olayını ata
    start_button.on_click(on_button_clicked)

    # Arayüz yerleşimini oluştur
    print("Eğitmek istediğiniz modelleri seçin:")
    ui_container = widgets.VBox(model_checkboxes + [start_button, output_area])

    # Arayüzü göster
    display(ui_container)
    print("✅ Arayüz başarıyla görüntülendi. Lütfen model seçimi yapın.")

# Arayüzü Görüntüle
display_training_ui()